In [1]:
import sys
import skimage
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import dask
import dask.array as da
import numpy as np
from dask.dot import dot_graph
from dask.distributed import Client, LocalCluster
from dask.distributed import Scheduler
from skimage.filters import threshold_otsu
from skimage.feature import blob_dog, blob_log, blob_doh
from skimage import data


sample = skimage.io.imread('/Users/nivethamahalakshmibalasamy/Documents/ECI-PolarScience/dask_stuff/seal/seal1.jpeg')
print sample.shape
print sample.dtype

(223, 223)
uint8


In [2]:
# Read images using delayed function into an image array
def array_images():
	custom_imread = dask.delayed(skimage.io.imread, pure = True)
	images = [custom_imread('/Users/nivethamahalakshmibalasamy/Documents/ECI-PolarScience/dask_stuff/seal/seal%d.jpeg' %i) for i in range(1,64)]
	#print images
	image_array = [da.from_delayed(i, sample.shape, sample.dtype) for i in images]
	sizes =  [j.shape for j in image_array]
	#print sizes
	stack = da.stack(image_array, axis = 0)
	print stack
	#print stack[0]
	# Combining chunks - A chunk consists of 10 images
	stack = stack.rechunk((10, 2000, 2000))
	print "After rechunking: "
	temp = stack
	stack.visualize()

	# Distribute array components over workers and centralized scheduler
	cluster = LocalCluster()
	client = Client(cluster)
	print client

	# Load the entire distributed array on the cluster (4 workers, 4 cores)
	stack = client.persist(stack)
	print "stack.mean().compute() is %d"%stack.mean().compute()

	# map the otsu thresholding function
	#print stack[0]
	stack = da.map_blocks(otsu_thresholding, stack, chunks = (10,2000,2000), dtype = sample.dtype)
	#stack = client.persist(stack)
	#stack = da.map_blocks(blob_detection, stack, chunks = (5,2000,2000), dtype = sample.dtype)
	stack = client.persist(stack)
	print "stack.mean().compute() is %d"%stack.mean().compute()
    
    
    
    


In [3]:
def otsu_thresholding(image):
	thresh = threshold_otsu(image)
	#print image.shape
	binary = image > thresh
	return binary

In [4]:
if __name__ == '__main__':
	array_images()

dask.array<stack, shape=(63, 223, 223), dtype=uint8, chunksize=(1, 223, 223)>
After rechunking: 


distributed.deploy.local - INFO - To start diagnostics web server please install Bokeh


<Client: scheduler='tcp://127.0.0.1:8786' processes=4 cores=4>
stack.mean().compute() is 70
stack.mean().compute() is 0
